In [20]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'/home/ubuntu/varios/skforecast'

In [21]:
from joblib import Parallel, delayed

# Define the function that returns two outputs
def compute(x):
    return x * x, x + x

# List of inputs
inputs = [1, 2, 3, 4, 5]

# Use Parallel and delayed to parallelize the function call
results = Parallel(n_jobs=-1)(delayed(compute)(i) for i in inputs)

# Extract the separate outputs
output1 = [result[0] for result in results]
output2 = [result[1] for result in results]

# Print the outputs
print("Output 1:", output1)
print("Output 2:", output2)


Output 1: [1, 4, 9, 16, 25]
Output 2: [2, 4, 6, 8, 10]


In [22]:
# simulate time series data
import numpy as np
import pandas as pd

np.random.seed(0)
n = 10
dates = pd.date_range(start='2020-01-01', periods=n)
data = np.random.randn(n)
y = pd.Series(data, index=dates)
y_2 = pd.Series(data, index=dates)

y_all = pd.concat([y, y_2])
y_all.drop_duplicates()


2020-01-01    1.764052
2020-01-02    0.400157
2020-01-03    0.978738
2020-01-04    2.240893
2020-01-05    1.867558
2020-01-06   -0.977278
2020-01-07    0.950088
2020-01-08   -0.151357
2020-01-09   -0.103219
2020-01-10    0.410599
dtype: float64

In [28]:
import inspect
from functools import wraps

def add_y_train_argument(func):
    @wraps(func)
    def wrapper(*args, y_train=None, **kwargs):
        # Call the original function with the given arguments, excluding y_train
        return func(*args, **kwargs)

    # Get the signature of the input function
    sig = inspect.signature(func)

    # Check if 'y_train' is in the function's parameters
    if "y_train" not in sig.parameters:
        # Update the signature of the new function to include 'y_train'
        new_params = list(sig.parameters.values()) + [
            inspect.Parameter("y_train", inspect.Parameter.KEYWORD_ONLY, default=None)
        ]
        wrapper.__signature__ = sig.replace(parameters=new_params)

    return wrapper

# Example usage
@add_y_train_argument
def example_function(x, z):
    return x + z

print(inspect.signature(example_function))  # Output: (x, z, *, y_train=None)


(x, z, *, y_train=None)


In [32]:
def example_function(x, z):
    return x + z

example_function = add_y_train_argument(example_function)
example_function(1, 2)

3

In [5]:
## rmsse

import numpy as np

train = np.array([
    [4,2,1,3,2],
    [400,200,100,300,200]
])
test = np.array([
    [2,0,4,1,1],
    [200,0,400,100,100]
])
forecast = np.array([
    [2,2,2,2,2],
    [200,200,200,200,200]
])

def rmsse(train, test, forecast):
    forecast_mse = np.mean((test-forecast)**2, axis=1)
    train_mse = [np.mean((np.diff(np.trim_zeros(row))**2)) for row in train]
    return np.sqrt(forecast_mse/train_mse)

print(f'RMSSE: {rmsse(train, test, forecast)}')

# Mase

def mase(train, test, forecast):
    forecast_mae = np.mean(np.abs(test-forecast), axis=1)
    train_mae = [np.mean(np.abs(np.diff(np.trim_zeros(row)))) for row in train]
    return (forecast_mae/train_mae)

print(f'mase: {mase(train, test, forecast)}')

RMSSE: [0.89442719 0.89442719]
mase: [0.8 0.8]
